# Revised Universal Soil Loss Equation Notebook

## Introduction

Soil erosion poses a growing threat to global food security, with up to 40% of the world’s topsoil already degraded — a figure projected to rise to 90% by 2050 (UNCCD, 2022). As population growth intensifies land pressure, the ability to model and manage soil loss becomes critical. The Revised Universal Soil Loss Equation (RUSLE) is a widely adopted empirical model for estimating average annual soil erosion caused by rainfall and surface runoff.

This notebook implements a GIS-based RUSLE workflow in Python using open-source libraries. Each factor in the RUSLE equation is calculated separately and transparently:

```
A = R × K × LS × C × P
```

Where:
- *A* = annual soil loss (tonnes/hectare/year)
- *R* = rainfall erosivity
- *K* = soil erodibility
- *LS* = slope length and steepness
- *C* = cover management
- *P* = support practices

## Calculating R

The initial data for R comes from the Centre for Environmental Data Analysis (CEDA) (https://data.ceda.ac.uk/badc/ukmo-midas-open/data/uk-daily-rain-obs), in this example rainfall point data data for the six counties Northern Ireland was used.

In [ ]:
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.plot import show
from gstools import Krige, Spherical
import tqdm
import os
import matplotlib.pyplot as plt

# Load and project point data
gdf = gpd.read_file(r"C:\GIS_Course\EGM721\Github conversion of RUSLE\RUSLE-in-Python\Data\R\NI_2021_Rain_Point.geojson")
gdf = gdf.to_crs(epsg=29903)

# Extract coords and rainfall
coords = np.array([(pt.x, pt.y) for pt in gdf.geometry])
values = gdf["RAINFALL"].values

# Create grid
grid_res = 1000
minx, miny, maxx, maxy = gdf.total_bounds
gridx = np.arange(minx, maxx + grid_res, grid_res)
gridy = np.arange(miny, maxy + grid_res, grid_res)
grid_xx, grid_yy = np.meshgrid(gridx, gridy)

# Define variogram model
model = Spherical(dim=2, len_scale=10000, var=np.var(values))

# Perform kriging with progress
krig = Krige(model, cond_pos=coords.T, cond_val=values)
print("Interpolating with kriging...")
field = np.full(grid_xx.shape, np.nan)

for i in tqdm.tqdm(range(len(gridy)), desc="Kriging rows"):
    row_x = grid_xx[i, :]
    row_y = grid_yy[i, :]
    points = np.vstack([row_x, row_y])
    field[i, :], _ = krig(points)

# Save rainfall raster to GeoTIFF
transform = from_origin(minx, maxy, grid_res, grid_res)
os.makedirs(r"C:\GIS_Course\EGM721\Github conversion of RUSLE\RUSLE-in-Python\Data\R", exist_ok=True)
rainfall_path = r"C:\GIS_Course\EGM721\Github conversion of RUSLE\RUSLE-in-Python\Data\R\rainfall_kriging.tif"
with rasterio.open(
    rainfall_path,
    "w",
    driver="GTiff",
    height=field.shape[0],
    width=field.shape[1],
    count=1,
    dtype="float32",
    crs="EPSG:29903",
    transform=transform,
) as dst:
    dst.write(np.flipud(field), 1)

# Calculate R-Factor
with rasterio.open(rainfall_path) as src:
    rainfall = src.read(1)
    profile = src.profile

r_factor = 38.46 + (3.48 * rainfall)
profile.update(dtype=rasterio.float32)

# Save R-Factor raster
r_factor_path = r"C:\GIS_Course\EGM721\Github conversion of RUSLE\RUSLE-in-Python\Data\R\r_factor.tif"
with rasterio.open(r_factor_path, "w", **profile) as dst:
    dst.write(r_factor.astype(np.float32), 1)

# Plot R-Factor raster
with rasterio.open(r_factor_path) as src:
    r_factor_data = src.read(1)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

plt.figure(figsize=(10, 8))
plt.imshow(r_factor_data, cmap='viridis', extent=extent)
plt.colorbar(label="R-Factor (Rainfall Erosivity)")
plt.title("Rainfall Erosivity Factor (R)")
plt.xlabel("Easting (m)")
plt.ylabel("Northing (m)")
plt.grid(False)
plt.show()
